In [12]:
import pandas as pd
import os
from bs4 import BeautifulSoup as bsoup
import calendar
import requests 

In [13]:
def get_date(website):
    response = requests.get('https://www.siliconvalley.com')
    content=bsoup(response.text,'html.parser')
    date=content.find('div',class_='date').get_text().strip().split('\n')[0].strip()
    date_month_year=date.split(',')[-1].strip().split()
    month_name_to_num={month: index for index, month in enumerate(calendar.month_abbr) if month}
    month_num=month_name_to_num[date_month_year[0]]
    if month_num < 10:
        month_num = '0'+str(month_num)
    date='' .join((z for z in date_month_year[1] if  z.isdigit()))
    year=date_month_year[-1]
    
    return date,year,month_num

def extract_headline_urls(url,dates,dict_store_values,date,final):
    
    response = requests.get(url)
    content=bsoup(response.text,'html.parser') # url
    all_data=content.find_all(['h3','h2','div'],class_=['entry-title','entry-title','entry-meta'])
    for i in range(len(all_data)):
       
        if all_data[i].name == 'h3':
            if all_data[i].find('span').get('class')[-1]=='metered':
                link=all_data[i].find_all('a')[-1].get('href')
                #head_line_urls.append(link) 
                link_date = link.split('.com')[-1].split('/')[3]
                dates.append(link_date)
                #headlines.append(all_data[i].find('span').get_text().strip())
                dict_store_values[all_data[i].find('span').get_text().strip()]=[link,link_date]

        else:
            if all_data[i].name == 'h2':
                if all_data[i].find('span').get('class')[-1]=='metered':
                    #headlines.append(all_data[i].find('span').get_text().strip())
                    #head_line_urls.append(all_data[i].find_all('a')[-1].get('href'))
                    dates.append(all_data[i+1].find('time').get_text().split()[1].split(',')[0])
                    dict_store_values[all_data[i].find('span').get_text().strip()] = [all_data[i].find_all('a')[-1].get('href'),
                                                                      all_data[i+1].find('time').get_text().split()[1].split(',')[0]]
                
    
    for i in list(dict_store_values.keys()):
        if dict_store_values[i][-1] == date:
            final[i] = dict_store_values[i][0]
            
    if dates[-1] ==  date:
        #do load more 
        url_next_page=content.find('a',class_='load-more').get('href')
        extract_headline_urls(url_next_page,dates,dict_store_values,date,final)
    else:
        return final
    
def get_article_data(url):
    article=[]
    res=requests.get(url).text
    content=bsoup(res,'html.parser')
    data=content.find('div',class_='body-copy')
    article_data= data.find_all(['p','li','h4','h3','h2','h1'])
    for i in article_data:
        article.append(i.get_text())
        
    return ' '.join(article)


In [14]:
def meta_data():
    pass

def extract(category_urls,article_data,dates,dict_store_values,date,final):
    for i in category_urls:
        final_dict=extract_headline_urls(i,dates,dict_store_values,date,final)
        
    headlines=list(final_dict.keys())
    urls=list(final_dict.values())
    
    if len(headlines) == 0:
        return 0,0
    
    for u in urls:
        article_data.append(get_article_data(u))
    
    return headlines,article_data

In [58]:
def main():
    website= 'https://www.siliconvalley.com'
    year=0
    date='22'
    month_num=0
    categories= ['Business',
    'Technology',
    'Commercial Real Estate',
    'Housing']
    category_urls=['https://www.siliconvalley.com/business/',
                   'https://www.siliconvalley.com/business/technology/',
                   'https://www.siliconvalley.com/tag/commercial-real-estate/',
                   'https://www.siliconvalley.com/tag/housing/']
    
    dict_store_values = {}
    dates=[]
    final={}
    article_data=[]
    
    #date,year,month_num = get_date(website)
    
    headline,article_data = extract(category_urls,article_data,dates,dict_store_values,date,final)
    
    if headline == 0:
        return -1
    
    json_data={'headlines':headline,"information":article_data}
    
    
    df=pd.DataFrame(json_data)
    df['date']=date

    return df

In [59]:
main()

,headlines,information,date
0,Single family residence sells in Saratoga for ...,"A 2,152-square-foot house built in 1962 has ch...",22
1,Sale closed in Pleasanton: $1.6 million for a ...,"A 1,919-square-foot house built in 1975 has ch...",22
2,VP Kamala Harris visits Silicon Valley to high...,Vice President Kamala Harris toured Silicon Va...,22
3,TikTok sues Montana over law banning the app,"By Amy Beth Hanson | Associated Press HELENA, ...",22
4,Two-bedroom home in Palo Alto sells for $2.8 m...,The historic property located in the 1400 bloc...,22
5,Downtown Berkeley apartments could replace off...,BERKELEY — A veteran Bay Area real estate firm...,22
6,Multiple verified accounts on Twitter share fa...,By Donie O’Sullivan and Jon Passantino | CNN A...,22
7,Single-family home in Los Gatos sells for $2 m...,The property located in the 200 block of More ...,22
8,"Supreme Court ducks fight over foie gras, leav...",The Supreme Court declined on Monday to take u...,22
9,Meta fined record $1.3 billion and ordered to ...,By KELVIN CHAN (AP Business Writer) The Europe...,22
